# Imports

In [1]:
import pandas as pd
import numpy as np
 
# TensorFlow
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, LSTM, Concatenate
from tensorflow.keras.layers import Dropout, Dense, Lambda
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam


# Scikit-learn
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

# Text preprocessing
import nltk
# nltk.download('punkt')
# nltk.download('wordnet')
from nltk.tokenize import word_tokenize
import re
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical

# Plots
import seaborn as sns
import matplotlib.pyplot as plt

# Misc.
import os
import joblib
import random
import time
from tqdm import tqdm_notebook as tqdm
import pretty_midi

SEED = 42
%matplotlib inline

In [2]:
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Load Data

### Load raw data

In [3]:
cols = ['Singer', 'Song Name', 'Lyrics']

df = pd.read_csv('datasets/lyrics_train_set.csv', names=cols)
df_test = pd.read_csv('datasets/lyrics_test_set.csv', names=cols)

#### Remove non rational lyrics

In [4]:
bad_song = df.query("Singer == 'darude'")
bad_song

,Singer,Song Name,Lyrics
315,darude,sandstorm,[instrumental] & du du dudududududuud & dududu...


In [5]:
df.drop(axis=0, index=bad_song.index, inplace=True)

### Add midi files' names

In [6]:
def add_midi_files(df):
    midis = list(os.listdir(r'./datasets/midi_files'))
    midis = {midi.lower()[:-4]: midi for midi in midis}    
    
    def combine_singer_song(singer, song):
        key = f'{singer} - {song}'.replace(' ', '_').lower()
        return midis[key] if key in midis else None
    
    df['Midi File'] = df.apply(lambda r: combine_singer_song(r['Singer'], r['Song Name']) ,axis=1)
    return df

In [7]:
df = add_midi_files(df)
df_test = add_midi_files(df_test)

#### check midi files

In [8]:
midis = np.concatenate([df['Midi File'].values, df_test['Midi File'].values])
midis.shape

(619,)

In [9]:
corrupted = []
for i, midi in enumerate(midis):
    try:
        midi = pretty_midi.PrettyMIDI(fr'./datasets/midi_files/{midi}')
        midi.remove_invalid_notes()
        del midi
    except Exception as e:
        print("%s\nerror readying midi file %s" % (e, midi))
        corrupted.append(i)

/home/liavba/.conda/envs/liav_env/lib/python3.7/site-packages/pretty_midi/pretty_midi.py:101: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  RuntimeWarning)


Could not decode key with 16 sharps and mode 1
error readying midi file David_Bowie_-_Lazarus.mid
Could not decode key with 1 flats and mode 255
error readying midi file Beastie_Boys_-_Girls.mid
data byte must be in range 0..127
error readying midi file Billy_Joel_-_Movin'_Out.mid
data byte must be in range 0..127
error readying midi file Billy_Joel_-_Pressure.mid
Could not decode key with 4 flats and mode 255
error readying midi file Dan_Fogelberg_-_Leader_of_the_Band.mid

error readying midi file Brian_McKnight_-_On_The_Down_Low.mid
data byte must be in range 0..127
error readying midi file Aaron_Neville_-_Tell_It_Like_It_Is.mid


In [10]:
bad_songs = df.iloc[corrupted]
bad_songs

,Singer,Song Name,Lyrics,Midi File
91,david bowie,lazarus,look up here i'm in heaven & i've got scars th...,David_Bowie_-_Lazarus.mid
115,beastie boys,girls,girls all i really want is girls & and in the ...,Beastie_Boys_-_Girls.mid
136,billy joel,movin' out,anthony works in the grocery store & savin h...,Billy_Joel_-_Movin'_Out.mid
143,billy joel,pressure,you have to learn to pace yourself & pressure ...,Billy_Joel_-_Pressure.mid
189,dan fogelberg,leader of the band,an only child alone and wild a cabinet maker's...,Dan_Fogelberg_-_Leader_of_the_Band.mid
513,brian mcknight,on the down low,maxine was 5'9'' & had a man and she didn't mi...,Brian_McKnight_-_On_The_Down_Low.mid
575,aaron neville,tell it like it is,if you want something to play with & go and fi...,Aaron_Neville_-_Tell_It_Like_It_Is.mid


In [11]:
df.drop(axis=0, index=bad_songs.index, inplace=True)

### Create Validation Dataset

In [12]:
msk = np.random.rand(len(df)) < 0.8

df_train = df[msk]
df_val = df[~msk]

# Preprocessing

## Lyrics preprocessing

In [13]:
from nltk.tokenize import word_tokenize

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    phrase = re.sub(r"in\'", "ing", phrase)
    phrase = re.sub(r"y\'all", "you all", phrase)
    phrase = re.sub(r"hiya", "hi you", phrase)
    
    # punctions
    regex = re.compile('[^a-zA-Z& ]')
    phrase = regex.sub('', phrase)
    
    return phrase

def preprocess_lyrics(data):
    data = decontracted(data)
    tokens = word_tokenize(data)
    data_arr = []
    
    for t in tokens:
        # Use only words, character combinations and numbers 
#         if not t.isalpha(): 
#             continue
            
        # Lower case word
        t = t.lower()
        
#         # Remove stop words
#         if t in sw: 
#             continue
        
        data_arr.append(t)
    
    
    return data_arr

In [14]:
df_train.iloc[1,2]

"you know i need your love & you've got that hold over me & long as i've got your love & you know that i'll never leave & when i wanted you to share my life & i had no doubt in my mind & and it's been you woman & right down the line & i know how much i lean on you & only you can see & the changes that i've been through & have left a mark on me & you've been as constant as a northern star & the brightest light that shines & it's been you woman right down the line & i just want to say this is my way & of tellin' you everything & i could never say before & yeah this is my way of tellin' you & that every day i'm lovin' you so much more & 'cause you believed in me through my darkest night & put somethin' better inside of me & you brought me into the light & threw away all those crazy dreams & i put them all behind & and it was you woman & right down the line & i just want to say this is my way of tellin' you everything & i could never say before & yeah this is my way of tellin' you & everyt

In [15]:
string = df_train.iloc[1,2]
tokenized_string = preprocess_lyrics(string)

def pretty_lyrics(tokenized_string):
    for token in tokenized_string:
        if token == '&':
            print('\n')
        else:
            print(token, end=' ')

pretty_lyrics(tokenized_string)

you know i need your love 

you have got that hold over me 

long as i have got your love 

you know that i will never leave 

when i wanted you to share my life 

i had no doubt in my mind 

and it is been you woman 

right down the line 

i know how much i lean on you 

only you can see 

the changes that i have been through 

have left a mark on me 

you have been as constant as a northern star 

the brightest light that shines 

it is been you woman right down the line 

i just want to say this is my way 

of telling you everything 

i could never say before 

yeah this is my way of telling you 

that every day i am loving you so much more 

cause you believed in me through my darkest night 

put something better inside of me 

you brought me into the light 

threw away all those crazy dreams 

i put them all behind 

and it was you woman 

right down the line 

i just want to say this is my way of telling you everything 

i could never say before 

yeah this is my way of telling y

In [16]:
stop_token = '$'
lyrics_train = df_train['Lyrics'].apply(lambda s: preprocess_lyrics(s)[:-1] + [stop_token])
lyrics_val = df_val['Lyrics'].apply(lambda s: preprocess_lyrics(s)[:-1] + [stop_token])

### Create embeddings

In [17]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lyrics_train)

In [18]:
lyrics_train = tokenizer.texts_to_sequences(lyrics_train)
lyrics_val = tokenizer.texts_to_sequences(lyrics_val)

In [19]:
EMBEDDING_FILE = './GoogleNews-vectors-negative300.bin'

if not os.path.isfile(EMBEDDING_FILE):
    !wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"
    !gzip -f -d GoogleNews-vectors-negative300.bin.gz

In [20]:
from gensim import models

embeddings_index = models.KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)
embed_size = 300
word_index = tokenizer.word_index
max_features = len(word_index) + 1

nb_words = len(word_index)
embedding_matrix = (np.random.rand(nb_words+1, embed_size) - 0.5) / 5.0

not_in_word2vec = 0
for word, i in word_index.items():
    if i >= max_features: continue
    if word in embeddings_index:
        embedding_vector = embeddings_index.get_vector(word)
        embedding_matrix[i] = embedding_vector
    else:
        not_in_word2vec += 1
        
print(f'{not_in_word2vec} out of {len(word_index)} has no embedings from word2vec')

414 out of 6160 has no embedings from word2vec


#### Trying one word to whole song but one word

In [ ]:
# train_x, train_y = [], []

# for lyric in lyrics:
#     for i in range(1, len(lyric)):
#         train_x.append(lyric[:i])
#         train_y.append(*lyric[i:i+1])
        
# train_x = pad_sequences(train_x)
# train_y = to_categorical(train_y)
# train_x.shape, train_y.shape

#### Trying sliding window of words

In [21]:
ast=np.lib.index_tricks.as_strided
def generate_sliding_window(arr, window_size=5, window_stride=1, last_window=False):
    last_window = 1 if last_window else 0
    arr = np.ascontiguousarray(arr)
    arr_len = arr.shape[0]
    s, = arr.strides
    windows_num = ((arr_len-window_size)//window_stride) + last_window
    
    return ast(arr, (windows_num, window_size), (s*window_stride, s))

In [22]:
def split_lyrics(lyrics, window_size=10):
    X, y = [], []
    for lyric in lyrics:
        X.append(generate_sliding_window(lyric, window_size))
        y.append(to_categorical(lyric[window_size:], num_classes=max_features))
        
    return X, y


In [23]:
window_size = 10
lyrics_train = split_lyrics(lyrics_train, window_size)
lyrics_val = split_lyrics(lyrics_val, window_size)

## Melody preprocessing

In [24]:
ast=np.lib.index_tricks.as_strided
def generate_sliding_window_2d(arr, window_size=5, window_stride=1, last_window=False):
    last_window = 1 if last_window else 0
    arr = np.ascontiguousarray(arr)
    l0, l1 = arr.shape
    s0, s1 = arr.strides
    windows_num = ((l0-window_size)//window_stride) + last_window
    
    return ast(arr, (windows_num, window_size, l1), (s0*window_stride, s0, s1))

In [25]:
def preprocess_melody_file_by_time(midi_name, fs=5, max_piano_val=100, fpw=4, window_size=10):
    pm = pretty_midi.PrettyMIDI(fr'./datasets/midi_files/{midi_name}')
    pm.remove_invalid_notes()
    
    # Sum all notes from all instruments
    piano_all = pm.get_piano_roll(fs=fs)
    piano_shape = piano_all.shape

    # Normalize by the number of played instruments in the same time and note
    counter = np.zeros(piano_shape)
    for inst in pm.instruments:
        curr_piano = inst.get_piano_roll(fs=fs)

        counter[:, :curr_piano.shape[1]] += (curr_piano > 0).astype(int)

    counter[counter == 0] = 1
    piano_all /= counter
    
    # Normalize by the maximum value of a note
    piano_all = piano_all / max_piano_val
    
    # Normalize by the number of played notes in the same time
    count_notes = (piano_all > 0).sum(axis=0)
    count_notes[count_notes == 0] = 1
    melody = piano_all.sum(axis=0) / count_notes
    del piano_all
    
    # 3 frames of 5fps equal to 0.6s ~ about one word
    melody_per_word = generate_sliding_window(melody[(melody > 0).argmax():], window_size=fpw, window_stride=(fpw*2)//5, last_window=True)
    melody_windows = generate_sliding_window_2d(melody_per_word, window_size=window_size, last_window=True)
    
    return melody_windows

In [26]:
def preprocess_melody_file_by_note(midi_name, fs=5, max_piano_val=100):
    pm = pretty_midi.PrettyMIDI(fr'./datasets/midi_files/{midi_name}')
    pm.remove_invalid_notes()
    
    # Sum all notes from all instruments
    piano_all = pm.get_piano_roll(fs=fs)
    piano_shape = piano_all.shape

    # Normalize by the number of played instruments in the same time and note
    counter = np.zeros(piano_shape)
    for inst in pm.instruments:
        curr_piano = inst.get_piano_roll(fs=fs)

        counter[:, :curr_piano.shape[1]] += (curr_piano > 0).astype(int)

    counter[counter == 0] = 1
    piano_all /= counter
    
    # Normalize by the maximum value of a note
    piano_all = piano_all / max_piano_val
    
    # Normalize by the number of played notes in the same time
    count_notes = (piano_all > 0).sum(axis=1)
    count_notes[count_notes == 0] = 1
    melody = piano_all.sum(axis=1) / count_notes
    
    return melody


In [27]:
midis = np.concatenate([df['Midi File'].values, df_test['Midi File'].values])
fs = 5

max_note_val = -1
min_frames = 9999
max_frames = -1
for midi in tqdm(midis):
    pm = pretty_midi.PrettyMIDI(fr'./datasets/midi_files/{midi}')
    pm.remove_invalid_notes()
    
    piano_roll = pm.get_piano_roll(fs=fs)
    if piano_roll.shape[1]:
        curr_len = piano_roll.shape[1]
        min_frames = min(min_frames, curr_len)
        max_frames = max(max_frames, curr_len)
        
    for inst in pm.instruments:
        piano_roll = inst.get_piano_roll(fs=fs)
        if piano_roll.shape[1]:
            curr_max_note = piano_roll.max()
            max_note_val = max(max_note_val, curr_max_note)
        
    del pm

print(f'The maximum note value is {max_note_val}')
print(f'The minimum song length is {min_frames}')
print(f'The maximum song length is {max_frames}')

/home/liavba/.conda/envs/liav_env/lib/python3.7/site-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


  0%|          | 0/612 [00:00<?, ?it/s]

/home/liavba/.conda/envs/liav_env/lib/python3.7/site-packages/pretty_midi/pretty_midi.py:101: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  RuntimeWarning)


KeyboardInterrupt: 

In [28]:
melody_train_slided = df_train['Midi File'].apply(preprocess_melody_file_by_time, fs=100, max_piano_val=762, fpw=100, window_size=10)
melody_val_slided = df_val['Midi File'].apply(preprocess_melody_file_by_time, fs=100, max_piano_val=762, fpw=100, window_size=10)

KeyboardInterrupt: 

In [29]:
melody_train_simple = df_train['Midi File'].apply(preprocess_melody_file_by_note, fs=100, max_piano_val=762)
melody_val_simple = df_val['Midi File'].apply(preprocess_melody_file_by_note, fs=100, max_piano_val=762)

In [30]:
def prepare_x_y_note(lyrics, melody):
    for i in range(len(lyrics[0])):
        window_n, window_size = lyrics[0][i].shape
        
        melody.iloc[i] = np.repeat([melody.iloc[i]], repeats=window_n * window_size).reshape(window_n, window_size, -1)
        
            
    lyrics_X = np.concatenate(lyrics[0])
    lyrics_y = np.concatenate(lyrics[1])
    melody = np.concatenate(melody.values)
            
    return (melody, lyrics_X), lyrics_y

In [31]:
train_data_simple = prepare_x_y_note(lyrics_train, melody_train_simple)
val_data_simple = prepare_x_y_note(lyrics_val, melody_val_simple)

In [ ]:
def prepare_x_y_time(lyrics, melody):
    for i in range(len(lyrics[0])):
        mel_len = melody.iloc[i].shape[0]
        lyr_len = lyrics[0][i].shape[0]
        
        if mel_len >= lyr_len:
            melody.iloc[i] = melody.iloc[i][:lyr_len, :, :]
        else:
            lyrics[0][i] = lyrics[0][i][:mel_len, :]
            lyrics[1][i] = lyrics[1][i][:mel_len, :]
            
    lyrics_X = np.concatenate(lyrics[0])
    lyrics_y = np.concatenate(lyrics[1])
    del lyrics
    melody = np.concatenate(melody.values)
            
    return (melody, lyrics_X), lyrics_y

In [ ]:
train_data_slided = prepare_x_y(lyrics_train, melody_train_slided)
val_data_slided = prepare_x_y(lyrics_val, melody_val_slided)

# Building the Lyrics Generator model

In [32]:
def init_model(window_size, melody_features):
    input_melody = Input(shape=(window_size, melody_features), name='melody')
    input_lyrics = Input(shape=(window_size,), name='lyrics')
    
    embd_lyrics = Embedding(max_features, 
                      embed_size, 
                      weights=[embedding_matrix],
                      input_length=window_size,
                      name='word_embd')(input_lyrics)
    
    merged = Concatenate(axis=2, name='merge')([embd_lyrics, input_melody])
    
    lstm = LSTM(100, return_sequences=True, dropout=0.5)(merged)
    lstm = LSTM(100, dropout=0.5)(lstm)

    X = Dense(100, activation="relu")(lstm)
    X = Dropout(0.5)(X)
    out = Dense(max_features, activation="softmax", name = 'out')(X)

    model = Model([input_melody, input_lyrics], out)
    
#     model.get_layer('embd').trainable = False

    model.compile(loss='categorical_crossentropy', optimizer=Adam())
    
    return model

In [33]:
model = init_model(10,100)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
lyrics (InputLayer)             [(None, 10)]         0                                            
__________________________________________________________________________________________________
word_embd (Embedding)           (None, 10, 300)      1848300     lyrics[0][0]                     
__________________________________________________________________________________________________
melody (InputLayer)             [(None, 10, 100)]    0                                            
__________________________________________________________________________________________________
merge (Concatenate)             (None, 10, 400)      0           word_embd[0][0]                  
                                                                 melody[0][0]                 

In [66]:
def init_simple(seq_len):
    inp = Input(shape=(seq_len,))
    
    embd = Embedding(max_features, 
                      embed_size, 
                      weights=[embedding_matrix],
                      input_length=seq_len,
                      name='word_embd')(inp)
    
    lstm = LSTM(100, return_sequences=True)(embd)
    lstm = LSTM(100)(lstm)

    X = Dense(100, activation="relu")(lstm)
    X = Dropout(0.5)(X)
    out = Dense(max_features, activation="softmax", name = 'out')(X)

    model = Model(inp, out)
    
#     model.get_layer('embd').trainable = False

    model.compile(loss='categorical_crossentropy', optimizer=Adam())
    
    return model

In [34]:
def get_callbacks(model_name):
    acc = 'val_loss'
    acc_mode = 'min'
    
    checkpoint = ModelCheckpoint(
                              fr'./models/{model_name}.h5', 
                              monitor=acc, 
#                               verbose=1, 
                              save_best_only=True, 
                              mode=acc_mode)
    earlystop = EarlyStopping(monitor=acc, mode=acc_mode, verbose=1, patience=6)
    reduceLR = ReduceLROnPlateau(monitor = 'val_loss', mode = 'min', patience = 5,
                            factor = 0.5, min_lr = 1e-6, verbose = 1)

    return [checkpoint, reduceLR, earlystop]

In [35]:
def train_model(model_name, model_gen, train_data, val_data, use_saved=False, params_dict=None):
    os.makedirs('./models', exist_ok=True)
    params = ''
    if params_dict is not None:
        params = '_'.join(f'{key}_{val}' for key,val in params_dict.items())
    model_name = model_gen.__name__[5:] + f'_{model_name}' + f'_{params}'
        
    if use_saved:
        history = joblib.load(fr'./models/{model_name}_history.sav')
    else:
        callbacks = get_callbacks(model_name)
        
        train_x, train_y = train_data
        
        model = model_gen(*train_x[0].shape[1:]) # melody size
        history = model.fit(
                            x=train_x,
                            y=train_y,
                            batch_size=params_dict['batch_size'],
                            epochs=params_dict['epochs'],
                            validation_data=val_data,
                            callbacks=callbacks,
                            verbose=1
                            )
        
        history = history.history
        joblib.dump(history, fr'./models/{model_name}_history.sav')
    
    model = load_model(fr'./models/{model_name}.h5')
    
    return model, history

In [69]:
model = init_simple(train_x.shape[1])
model.summary()

NameError: name 'train_x' is not defined

In [70]:
params_dict = {'batch_size': 32, 'epochs': 5}
model, history= train_model(init_simple, train_data, val_data, use_saved=False, params_dict=params_dict)

TypeError: init_simple() takes 1 positional argument but 2 were given

In [37]:
params_dict = {'batch_size': 32, 'epochs': 20}
model, history= train_model('whole', init_model, train_data_simple, val_data_simple, use_saved=False, params_dict=params_dict)

Epoch 1/20
4499/4499 [==============================] - 81s 16ms/step - loss: 5.8421 - val_loss: 5.0701
Epoch 2/20
4499/4499 [==============================] - 66s 15ms/step - loss: 5.1992 - val_loss: 4.8369
Epoch 3/20
4499/4499 [==============================] - 77s 17ms/step - loss: 4.9348 - val_loss: 4.7561
Epoch 4/20
4499/4499 [==============================] - 77s 17ms/step - loss: 4.7859 - val_loss: 4.7425
Epoch 5/20
4499/4499 [==============================] - 77s 17ms/step - loss: 4.6453 - val_loss: 4.7311
Epoch 6/20
4499/4499 [==============================] - 77s 17ms/step - loss: 4.5675 - val_loss: 4.7332
Epoch 7/20
4499/4499 [==============================] - 77s 17ms/step - loss: 4.4905 - val_loss: 4.7547
Epoch 8/20
4499/4499 [==============================] - 77s 17ms/step - loss: 4.4507 - val_loss: 4.7531
Epoch 9/20
4499/4499 [==============================] - 77s 17ms/step - loss: 4.3979 - val_loss: 4.7887
Epoch 10/20
4499/4499 [==============================] - 77s 17m

In [142]:
params_dict = {'batch_size': 32, 'epochs': 20}
model, history= train_model('slided', init_model, train_data_slided, val_data_slided, use_saved=False, params_dict=params_dict)

Epoch 1/20
4273/4273 [==============================] - 75s 17ms/step - loss: 5.8492 - val_loss: 5.0854
Epoch 2/20
4273/4273 [==============================] - 60s 14ms/step - loss: 5.2188 - val_loss: 4.8665
Epoch 3/20
4273/4273 [==============================] - 72s 17ms/step - loss: 4.9603 - val_loss: 4.7791
Epoch 4/20
4273/4273 [==============================] - 72s 17ms/step - loss: 4.7815 - val_loss: 4.7518
Epoch 5/20
4273/4273 [==============================] - 72s 17ms/step - loss: 4.6575 - val_loss: 4.7285
Epoch 6/20
4273/4273 [==============================] - 72s 17ms/step - loss: 4.5703 - val_loss: 4.7296
Epoch 7/20
4273/4273 [==============================] - 72s 17ms/step - loss: 4.4919 - val_loss: 4.7443
Epoch 8/20
4273/4273 [==============================] - 72s 17ms/step - loss: 4.4269 - val_loss: 4.7891
Epoch 9/20
4273/4273 [==============================] - 72s 17ms/step - loss: 4.3919 - val_loss: 4.7714
Epoch 10/20
4273/4273 [==============================] - 72s 17m

# Sample Lyrics by Seed

In [72]:
def generate_song(model, seed, window_size, stop_token, tokenizer, max_len):
    stop_token = tokenizer.word_index[stop_token]
    
    def get_next_word(seed):
        probs = model(seed, training=False)
        chosen_idx = np.random.choice(range(0, max_features), p=probs[0])
        chosen_word = tokenizer.sequences_to_texts([[chosen_idx]])[0]
        
        return chosen_idx, chosen_word
    
    
    seed = preprocess_lyrics(seed)
    song = seed.copy()
    seed = ' '.join(seed)
    seed = tokenizer.texts_to_sequences([seed])
    seed = pad_sequences(seed, maxlen=window_size)

    i = 0
    idx, word = get_next_word(seed)
    while word != stop_token and i < max_len:
        song.append(word)
        i+=1
        seed = np.concatenate([seed[:,1:], [[idx]]], axis=1)
        idx, word = get_next_word(seed)
    
    return song    

In [ ]:
song_max_length = 2000
seed = 'Smack that all on the floor'
song = generate_song(model, seed, window_size, stop_token, tokenizer, song_max_length)
pretty_lyrics(song)

In [52]:
df_test

,Singer,Song Name,Lyrics,Midi File
0,the bangles,eternal flame,close your eyes give me your hand darling & do...,The_Bangles_-_Eternal_Flame.mid
1,billy joel,honesty,if you search for tenderness & it isn't hard t...,Billy_Joel_-_Honesty.mid
2,cardigans,lovefool,dear i fear we're facing a problem & you love ...,Cardigans_-_Lovefool.mid
3,aqua,barbie girl,hiya barbie & hi ken! & do you want to go for ...,Aqua_-_Barbie_Girl.mid
4,blink 182,all the small things,all the small things & true care truth brings ...,Blink_182_-_All_the_Small_Things.mid


In [47]:
def generate_song_with_melody(model, model_type, lyrics_seed, midi_name, window_size, tokenizer, max_len, fpw=100, stop_token='$'):
    stop_token_idx = tokenizer.word_index[stop_token]
    if model_type == 'slided':
        melody = preprocess_melody_file_by_time(midi_name, fs=100, max_piano_val=762, fpw=fpw, window_size=window_size)
    else:
        melody = preprocess_melody_file_by_note(midi_name, fs=100, max_piano_val=762)
        melody = np.repeat([melody], repeats=window_size, axis=0)
        melody = np.expand_dims(melody, axis=0)
    
    def get_next_word(lyrics_seed, melody_seed):
        if melody_seed is None:
            return stop_token_idx, stop_token
        try:
            probs = model((melody_seed, lyrics_seed), training=False)
            probs_v = np.array(probs[0])
            chosen_idx = np.random.choice(range(0, max_features), p=probs_v)
            chosen_word = tokenizer.sequences_to_texts([[chosen_idx]])[0]
            
        except Exception as e:
            print(e)
            return stop_token_idx, stop_token
        
        return chosen_idx, chosen_word
    
    def get_melody_seed(word_cnt):
        if model_type == "whole":
            return melody
        
        if word_cnt - window_size >= melody.shape[0]:
            return None
        
        if word_cnt < window_size:
            melody_seed = np.concatenate([np.zeros((window_size-word_cnt,fpw)),melody[0][:word_cnt,:fpw]])
        else:
            window_idx = word_cnt-window_size
            
            if window_idx >= melody.shape[0]: # Melody has ended
                return None
            
            melody_seed = melody[window_idx]
        
        return np.expand_dims(melody_seed, axis=0)
    
    
    lyrics_seed = preprocess_lyrics(lyrics_seed)
    song = lyrics_seed.copy()
    lyrics_seed = ' '.join(lyrics_seed)
    lyrics_seed = tokenizer.texts_to_sequences([lyrics_seed])
    word_cnt = len(lyrics_seed[0])
    lyrics_seed = pad_sequences(lyrics_seed, maxlen=window_size)

    melody_seed = get_melody_seed(word_cnt)
    idx, word = get_next_word(lyrics_seed, melody_seed)
    
    while idx != stop_token_idx and word_cnt < max_len:
        song.append(word)
        word_cnt+=1
        lyrics_seed = np.concatenate([lyrics_seed[:,1:], [[idx]]], axis=1)
        melody_seed = get_melody_seed(word_cnt)
        idx, word = get_next_word(lyrics_seed, melody_seed)
    
    return song    

In [48]:
def apply_gen_song(midi_name, model_type, lyrics_seed,  window_size=10, song_max_length=100,):
    print(f'\n[Song: {midi_name}, seed: {lyrics_seed}]\n')
    song = generate_song_with_melody(model, model_type, lyrics_seed, midi_name, window_size, tokenizer, song_max_length, fpw=100)
    pretty_lyrics(song)
    print('\n')


In [49]:
df_test['Midi File'].apply(lambda midi_name: apply_gen_song(midi_name,'whole', 'Hello'))


[Song: The_Bangles_-_Eternal_Flame.mid, seed: Hello]

hello 

cause the lovesick show come there to time 

tender ai not all easy i all settle 

bailamos 


[Song: Billy_Joel_-_Honesty.mid, seed: Hello]

hello 

nothing lie before 

baby i believe 

sentimental thing after summer 

niggas are when them wrong built lost your inside 

nothing gets the waking 

hold some still in an fall we know 

we are born 

anyone feels fire with the soldier x 

the glitter so properly 

it is the plan we are singing on for an willow bill of by come and happier 

dont ol nothing me along i will say 

i am all the eyes day where it is my wan na see 

do not reach 


[Song: Cardigans_-_Lovefool.mid, seed: Hello]

hello 

got up on the action and long till good care alone 

i do not know over oh 

the invisible could tell me forget here 

more through we have mine 

playing so weeks this suppper even shoes 

but i would gon na all it that kick in my love 

if i want to say and i am are girl 

you have b

0    None
1    None
2    None
3    None
4    None
Name: Midi File, dtype: object

In [51]:
df_test.apply(lambda r: apply_gen_song(r['Midi File'], 'whole', r['Lyrics'].split(' ')[0]), axis=1)


[Song: The_Bangles_-_Eternal_Flame.mid, seed: close]

close again 

now the dust reflecting run i more it around 

girl find my name i want me 

oh we say you wanted the strong is picking 

he like i do it do what make the so cash get san park 

to known that getting 

still leaves to just make me let me ever help you 

i know the will such life on where long 


[Song: Billy_Joel_-_Honesty.mid, seed: if]

if i move in after gave 

on the half is see 

you come 

the woman reaches night long down and a rum 

you still that hanging out and slip the fucking sky 

never just remember what my it is heaven 

oh when a running unfurled 

just hope you can not feed me a take them away 

just close 

santa soul with 

oh it is it up i really know more 

and nothing song ooh change 

how never me 

it is to so too in the set 




[Song: Cardigans_-_Lovefool.mid, seed: dear]

dear and rustles 

she 

word is around we listen to why 

and are someone 

lord you wan na do 

you wan na go 

the dad

0    None
1    None
2    None
3    None
4    None
dtype: object